## Import Pandas library and read CSV DataSet


In [177]:
import pandas as pd
df = pd.read_csv('pcr_dataset.csv')

In [178]:
df




,state,state_name,state_fips,fema_region,overall_outcome,date,new_results_reported,total_results_reported,geocoded_state
0,AL,Alabama,1,Region 4,Negative,2020/03/01,96,96,NaN
1,AL,Alabama,1,Region 4,Positive,2020/03/01,16,16,NaN
2,AL,Alabama,1,Region 4,Negative,2020/03/02,72,168,NaN
3,AL,Alabama,1,Region 4,Positive,2020/03/02,6,22,NaN
4,AL,Alabama,1,Region 4,Negative,2020/03/03,94,262,NaN
...,...,...,...,...,...,...,...,...,...
186844,WY,Wyoming,56,Region 8,Negative,2023/04/20,119,1436045,NaN
186845,WY,Wyoming,56,Region 8,Positive,2023/04/20,10,137792,NaN
186846,WY,Wyoming,56,Region 8,Inconclusive,2023/04/21,0,4013,NaN
186847,WY,Wyoming,56,Region 8,Negative,2023/04/21,28,1436073,NaN


## PCR Data Analysis for the following key metrics

#### Find the total number of PCR tests performed as of yesterday in the United States. Metric 1

In [179]:
total_no_of_pcr_tests=df['new_results_reported'].sum()
total_no_of_pcr_tests

1038952225

#### The 7-day rolling average number of new cases per day for the last 30 days. Metric 2
#### Filter Positive cases , calculate rolling avg for 7 days for the last 30 days

In [180]:
positive_cases_df= df[df['overall_outcome'] == 'Positive']

In [184]:
avg_number_of_new_cases_per_day = positive_cases_df.set_index('date')['new_results_reported'].rolling(window=7).mean().tail(30)

In [185]:
avg_number_of_new_cases_per_day

date
2023/03/23    17.428571
2023/03/24    17.714286
2023/03/25    18.000000
2023/03/26    18.714286
2023/03/27    17.000000
2023/03/28    16.285714
2023/03/29    13.857143
2023/03/30    15.142857
2023/03/31    14.714286
2023/04/01    14.571429
2023/04/02    13.571429
2023/04/03    14.142857
2023/04/04    13.285714
2023/04/05    12.857143
2023/04/06    11.571429
2023/04/07    12.428571
2023/04/08    12.714286
2023/04/09    13.428571
2023/04/10    13.857143
2023/04/11    16.285714
2023/04/12    16.714286
2023/04/13    17.571429
2023/04/14    16.285714
2023/04/15    17.000000
2023/04/16    16.714286
2023/04/17    14.285714
2023/04/18    12.857143
2023/04/19    13.714286
2023/04/20    12.142857
2023/04/21    10.857143
Name: new_results_reported, dtype: float64

#### Sum of Total Average Rate for positive case 

In [186]:
avg_number_of_new_cases_per_day.sum()/30


14.857142857142858

#### The 10 states with the highest test positivity rate (positive tests / tests performed) for tests performed in the last 30 days. Metric 3

#### Convert date string to datetime and find maximum date

In [158]:
df['date'] = pd.to_datetime(df['date'])
max_date = df['date'].max()

In [159]:
max_date

Timestamp('2023-04-21 00:00:00')

#### Filter last 30 days for each state using df

In [160]:
df = df[df['date'] >= max_date - pd.Timedelta(days=30)]

In [161]:
df

,state,state_name,state_fips,fema_region,overall_outcome,date,new_results_reported,total_results_reported,geocoded_state
3336,AL,Alabama,1,Region 4,Inconclusive,2023-03-22,0,18746,NaN
3337,AL,Alabama,1,Region 4,Negative,2023-03-22,3816,7905708,NaN
3338,AL,Alabama,1,Region 4,Positive,2023-03-22,211,1284147,NaN
3339,AL,Alabama,1,Region 4,Inconclusive,2023-03-23,3,18749,NaN
3340,AL,Alabama,1,Region 4,Negative,2023-03-23,4038,7909746,NaN
...,...,...,...,...,...,...,...,...,...
186844,WY,Wyoming,56,Region 8,Negative,2023-04-20,119,1436045,NaN
186845,WY,Wyoming,56,Region 8,Positive,2023-04-20,10,137792,NaN
186846,WY,Wyoming,56,Region 8,Inconclusive,2023-04-21,0,4013,NaN
186847,WY,Wyoming,56,Region 8,Negative,2023-04-21,28,1436073,NaN


#### To find Positive Rate , agg the count for postive cases by state and sum of pcr results by state.

In [162]:
positive_tests= df[df['overall_outcome'] == 'Positive'].groupby('state_name').size().reset_index(name='count')

In [163]:
len(positive_tests)

54

In [164]:
total_tests_performed = df[["new_results_reported","state_name","overall_outcome"]].groupby('state_name')['new_results_reported'].sum().reset_index(name='count')

In [165]:
len(total_tests_performed)

54

#### JOIN positive tests count and total tests performed by state 

In [166]:
merge_df = pd.merge(positive_tests,total_tests_performed , on='state_name', how='inner')

#### Calculate Positive Rate = positive tests/total test perfomed

In [167]:
rate_df = merge_df.groupby('state_name').agg({'count_x': 'sum', 'count_y': 'sum'})
positive_rate=rate_df['count_x'] / rate_df['count_y']

In [168]:
percent_positive_rate = positive_rate.apply(lambda x: '{:.2%}'.format(x))

In [169]:
positive_rate.sum()


0.4887053126290494

#### Using sort to find the top 10 states with highest postive rate

In [170]:
sort_df=percent_positive_rate.sort_values(ascending=False).reset_index().head(10)

In [171]:
sort_df

,state_name,0
0,U.S. Virgin Islands,41.67%
1,Guam,1.42%
2,North Dakota,0.64%
3,Wyoming,0.61%
4,Iowa,0.51%
5,Vermont,0.29%
6,Montana,0.28%
7,Nebraska,0.21%
8,Delaware,0.20%
9,South Dakota,0.20%
